In [1]:
from pyspark import SparkContext, SparkConf
from operator import add

conf = SparkConf().setAppName("Movies")
sc = SparkContext(conf = conf) 

from pyspark.sql import SQLContext
sqlContext = SQLContext(sc)


from pyspark.sql.types import *

In [2]:
df_users = sc.textFile('./moviedata/users.csv')
df_movies = sc.textFile('./moviedata/movies.csv')
df_rating = sc.textFile('./moviedata/rating.csv')
# df = sqlContext.read.format('com.databricks.spark.csv').load('./moviedata/rating.csv')

In [3]:
def select_age(p):
    return int(p[1])>=10 and int(p[1])<=18

In [4]:
#User between 10 and 18
possible_userid = df_users.map(lambda x: x.split(",")).filter(select_age).map(lambda x: (x[0],  ["epsilon"]))
#    map(lambda y: (int(y[1]),float(y[2])))
# .\
#     map(lambda x: (x[0], x[1])

In [11]:
possible_userid.count()

53

In [12]:
#p[7] - children genre
def select_genre(p):
    return int(p[7])==1

In [13]:
#movieIDs for children :|
possible_movieid = df_movies.map(lambda x: x.split(",")).filter(select_genre).map(lambda x: (x[0], ["epsilon"]))
#     reduce(lambda x,y:(x+y))

In [14]:
possible_movieid.take(5)

[(u'304', ['epsilon']),
 (u'308', ['epsilon']),
 (u'314', ['epsilon']),
 (u'374', ['epsilon']),
 (u'377', ['epsilon'])]

In [15]:
userid_movieid = df_rating.map(lambda x: x.split(",")).map(lambda x: (x[0], [(x[1])]))
movieid_userid = userid_movieid.map(lambda x: (x[1], x[0])) 

In [16]:
userid_movieid.take(10)

[(u'253', [u'97']),
 (u'284', [u'269']),
 (u'106', [u'526']),
 (u'121', [u'180']),
 (u'62', [u'86']),
 (u'291', [u'418']),
 (u'84', [u'1033']),
 (u'293', [u'380']),
 (u'207', [u'58']),
 (u'109', [u'97'])]

In [17]:
f = userid_movieid + possible_userid


In [18]:
f.take(10)

[(u'253', [u'97']),
 (u'284', [u'269']),
 (u'106', [u'526']),
 (u'121', [u'180']),
 (u'62', [u'86']),
 (u'291', [u'418']),
 (u'84', [u'1033']),
 (u'293', [u'380']),
 (u'207', [u'58']),
 (u'109', [u'97'])]

In [19]:
def find_epsilon(p):
    for i in p[1]:
        if(i == "epsilon"):
            return True 
    return False

In [20]:
#g - Group data by userID, epsilon is present in the movielist of users of age between 10 and 18
g = f.reduceByKey(lambda x, y: x+y).filter(find_epsilon)

In [21]:
#h - Records in which users are between 10 and 18
#h - MovieID - UserID
h = g.flatMap(lambda x: ((i, [x[0]]) for i in x[1] if i != "epsilon"))

In [22]:
h.take(13)

[(u'252', [u'674']),
 (u'763', [u'674']),
 (u'300', [u'674']),
 (u'118', [u'674']),
 (u'50', [u'674']),
 (u'685', [u'674']),
 (u'282', [u'674']),
 (u'929', [u'674']),
 (u'25', [u'674']),
 (u'304', [u'674']),
 (u'127', [u'674']),
 (u'405', [u'674']),
 (u'315', [u'674'])]

In [ ]:
# for i in g:
#     check_movies = i.map(lambda x: (i[1], 0)
#     check_movies + 
p = possible_movieid + h

In [ ]:
p.take(10)

In [ ]:
#Group users based on the movie they rated and then choose only children movies
r = p.reduceByKey(lambda x, y: x + y).filter(find_epsilon)

In [ ]:
r.collect()

In [ ]:
#Generate UserID - MovieID list from `r`
q = r.flatMap(lambda x: ((i, [x[0]]) for i in x[1] if i != "epsilon")).reduceByKey(lambda x,y: x+ y)
q.map(lambda x: x[0]).collect()

In [23]:
nonchildren_movies = df_movies.map(lambda x: x.split(",")).filter(lambda x: int(x[7])==0).map(lambda x: (x[0], ["epsilon"]))
#     reduce(lambda x,y:(x+y))

In [ ]:
nonchildren_movie_raters = (nonchildren_movies + )

In [10]:
nonchildren_movies.count()

1498